In [9]:
import tensorflow as tf

print(f"TensorFlow version: {tf.__version__}")
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

# Test GPU usage
if tf.config.list_physical_devices('GPU'):
    print("✓ GPU detected and available")
    with tf.device('/GPU:0'):
        a = tf.constant([1.0, 2.0, 3.0])
        b = tf.constant([4.0, 5.0, 6.0])
        c = tf.add(a, b)
        print(f"GPU computation result: {c}")
else:
    print("❌ No GPU available - using CPU")

TensorFlow version: 2.10.1
Built with CUDA: True
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
✓ GPU detected and available
GPU computation result: [5. 7. 9.]


In [10]:
'''
This ipynb file is adapted from the previous ResNet training script
Author: Jason Niow
DtaeL 
'''

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import pathlib

print(f'tensorflow version: {tf.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')
print(f'seaborn version: {sns.__version__}')

# check tensorflow GPU device support
if len(tf.config.list_physical_devices('GPU')) > 0:
    print('GPU present')
else:
    print('GPU absent')

# paths to load datasets from
train_store_path = 'C:/Users/UserAdmin/Desktop/Jason - Signal Classification/AI Models/Data/train_datasets/train_real_data_labelled'
test_store_path = 'C:/Users/UserAdmin/Desktop/Jason - Signal Classification/AI Models/Data/test_datasets/test_jul22_real'

# convert to pathlib Path objects
train_dir = pathlib.Path(train_store_path)
test_dir = pathlib.Path(test_store_path)

# get list of datasets paths in dir
train_ds_paths = sorted(list(train_dir.glob('*.csv')))
test_ds_paths = sorted(list(test_dir.glob('*.csv')))


# extract classification target from file names
train_ds_type = np.array([x.parts[-1].split('_')[:2] for x in train_ds_paths])
test_ds_type = np.array([x.parts[-1].split('_')[:2] for x in test_ds_paths])

# Get list of classification labels of dataset e.g. 8CPSK, FM, 16qam
train_ds_mod = [s.upper() for s in train_ds_type[:,0]]
test_ds_mod = [s.upper() for s in test_ds_type[:,0]]

# Get list of classification frequency
train_ds_freq = [s.upper() for s in train_ds_type[:, 1]]
test_ds_freq = [s.upper() for s in test_ds_type[:, 1]]

# generate signal type tags
known_signal_tags = {'16QAM', '8CPSK', 'FM'}
signal_tags = {'16QAM': 0, '8CPSK': 1, 'FM': 2}
# signal_tags = {k : i for i, k in enumerate(np.unique(sorted([s.upper() for s in train_ds_mod] + ['UNKNOWN'])))}

print(signal_tags)


tensorflow version: 2.10.1
pandas version: 2.0.3
numpy version: 1.25.1
seaborn version: 0.13.2
GPU present
{'16QAM': 0, '8CPSK': 1, 'FM': 2}


In [12]:

# load the dataset(s)

# load dataset information
specs = []
datasets = []

for dataset_paths in [train_ds_paths, test_ds_paths]:
    temp_ds = []
    temp_specs = []

    for path in dataset_paths:
        print(f'loading {path}...', end=' ')

        # load dataset details - Sampling frequency, Number of Samples, Number of Records
        df_spec = pd.read_csv(path, nrows=10, header=None, index_col=0, names=['info'])
        df_spec = df_spec.drop(['Version', 'DateTime', 'TimestampOffset', 'TriggerPosition', 'FastFrameID', 'IDInFastFrame', 'TotalInFastFrame'], axis=0).astype('int')

        temp_specs.append(df_spec)

        # load data, strip unnecessary bits out - I/Q data
        df = pd.read_csv(path, skiprows=10, names=['I', 'Q'])

        df = df.loc[~df['I'].isin(['TimestampOffset', 'TriggerPosition', 'FastFrameID', 'IDInFastFrame', 'TotalInFastFrame'])]
        df['I'] = df['I'].astype('float')

        print(f'loaded')

        temp_ds.append(df)

    datasets.append(temp_ds)
    specs.append(temp_specs)

print('done.')
        


loading C:\Users\UserAdmin\Desktop\Jason - Signal Classification\AI Models\Data\train_datasets\train_real_data_labelled\16qam_f270M_200k_10_chunk_1.csv... loaded
loading C:\Users\UserAdmin\Desktop\Jason - Signal Classification\AI Models\Data\train_datasets\train_real_data_labelled\16qam_f270M_200k_10_chunk_10.csv... loaded
loading C:\Users\UserAdmin\Desktop\Jason - Signal Classification\AI Models\Data\train_datasets\train_real_data_labelled\16qam_f270M_200k_10_chunk_11.csv... loaded
loading C:\Users\UserAdmin\Desktop\Jason - Signal Classification\AI Models\Data\train_datasets\train_real_data_labelled\16qam_f270M_200k_10_chunk_13.csv... loaded
loading C:\Users\UserAdmin\Desktop\Jason - Signal Classification\AI Models\Data\train_datasets\train_real_data_labelled\16qam_f270M_200k_10_chunk_14.csv... loaded
loading C:\Users\UserAdmin\Desktop\Jason - Signal Classification\AI Models\Data\train_datasets\train_real_data_labelled\16qam_f270M_200k_10_chunk_16.csv... loaded
loading C:\Users\UserAd

In [13]:
import numpy as np

def rms_normalize(iq, target_rms=1.0, eps=1e-12, remove_dc=True):
    """
    iq: complex64/complex128 numpy array, shape (N,)
    target_rms: desired RMS after scaling
    """
    x = iq.astype(np.complex64, copy=False)
    if remove_dc:
        x = x - x.mean()
    rms = np.sqrt(np.mean(np.abs(x)**2) + eps)
    return x * (target_rms / rms)

In [14]:

# split dataset(s) into records, extract test dataset
processed = []

# number of test records to extract
ntest = 100
rlength = 1024
nrecords = 1
nsamples = 10000

for h, dataset in enumerate(datasets): # loops through training, then testing
    if h == 0:
        dataset_type = 'TRAINING'
    else: dataset_type = 'TESTING'
    temp_processed = []
    specs_df = specs[h]

    print(f'\nType\t\tLocation\tTotal Records\tSamples/Record')
    # Loops through each data point in the dataset
    for i in range(len(dataset)):
        # nrecords = specs_df[i].loc['NumberRecords']['info'] if dataset_type == 'TRAINING' else 400 ### wtf is this
        # nrecords = specs_df[i].loc['NumberRecords']['info']
        # nsamples = specs_df[i].loc['NumberSamples']['info']

        ds_length = dataset[i].shape[0]

        # make life easier
        ds_mod = train_ds_mod if dataset_type == 'TRAINING' else test_ds_mod
        ds_freq = train_ds_freq if dataset_type == 'TRAINING' else test_ds_freq

        # sanity check
        print(f'{ds_mod[i]:<13}\t{ds_freq[i]:<15}\t{nrecords:<7}\t\t{nsamples:<7}')

        # loop through dataset to split
        for j in range(nrecords):
            # extract sample length worth of samples for each record, then transpose for easier access later
            # record = dataset[i].iloc[(nsamples * j):(nsamples * (j+1))].values.T
            iq = dataset[i].iloc[(nsamples * j):(nsamples * (j+1))].values
            iq_complex = iq[:, 0] + 1j * iq[:, 1]
            iq_rms = rms_normalize(iq_complex, target_rms=1.0, remove_dc=False).T
            i_rms = iq_rms.real.reshape((1, -1))
            q_rms = iq_rms.imag.reshape((1, -1))
            record = np.vstack((i_rms, q_rms))
            # print(f"Shape of record: {record.shape}")
            # pad shorter records with random padding to rlength
            if nsamples < rlength:
                print(f"i: {i} j : {j} Sample length {nsamples} is lesser than {rlength}")
                # deterine pad amount
                pad_length = rlength - nsamples
                lpad_length = np.random.randint(0, pad_length+1)
                rpad_length = pad_length - lpad_length

                # generate pad
                lpad = np.zeros((2, lpad_length))
                rpad = np.zeros((2, rpad_length))

                # concatenate pad
                record = np.concatenate([lpad, record, rpad], axis=1)

            # truncate longer records to rlength
            elif nsamples > rlength:
                # print(f"i: {i} j : {j} Sample length {nsamples} is greater than {rlength}")
                record = record[:,:rlength]

            # add processed record to list
            signal_tag = signal_tags.get(ds_mod[i], 3) # 3 is for proxy OOD samples
            # signal_tag = signal_tags[ds_mod[i].upper()]
            temp_processed.append([ds_mod[i], signal_tag, ds_freq[i], record])

    processed.append(temp_processed)

# convert list into dataframes for later use, randomise, extract test records
df_train = pd.DataFrame(processed[0], columns=['signal_type', 'tag', 'location', 'record']).sample(frac=1, random_state=42)
df_test = pd.DataFrame(processed[1], columns=['signal_type', 'tag', 'location', 'record']).sample(frac=1, random_state=42)

# print dataset statistics
print(f'\n{"Stats":^30}')
print(f'Dataset\tLength\tRecords/Sample')
print(f'Train\t{df_train.shape[0]:<5}\t{df_train["record"].iloc[0].shape[1]}')
print(f'Test\t{df_test.shape[0]:<5}\t{df_train["record"].iloc[0].shape[1]}')


# define one hot encode function
def one_hot(arr, n_cat):
    output = []
    for n in arr:
        if n == 3: # for proxy OOD samples
            result = np.zeros(n_cat)
        else: # for known classes
            result = np.zeros(n_cat)
            result[n] = 1

        output.append(result)

    return np.array(output, dtype=int)

# extract train and test data
X_train = np.concatenate(df_train['record'].values).reshape((df_train.shape[0], 2, rlength, 1))
y_train = one_hot(df_train['tag'].values, len(signal_tags))
y_ood = df_train['tag'] == 3



Type		Location	Total Records	Samples/Record
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F270M          	1      		10000  
16QAM        	F

In [15]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, concatenate, Flatten, Dense, Reshape, Conv2DTranspose

class Autoencoder(Model):
    def __init__(self, input_shape, latent_dim=64, output_shape=None):
        super().__init__()
        self.encoder = self.create_encoder(input_shape, latent_dim = 64)
        self.decoder = self.create_decoder(latent_dim, output_shape=input_shape)

    def create_encoder(self, input_shape, latent_dim):
        input_layer = tf.keras.Input(shape=input_shape)
        # res stacks
        x = self.res_stack(input_layer, 512)
        x = self.res_stack(x, 256)
        x = self.res_stack(x, 128)
        x = self.res_stack(x, 64)
        x = self.res_stack(x, 32)
        x = self.res_stack(x, 16)
        
        # Flatten and create latent representation
        x = Flatten()(x)
        latent = Dense(latent_dim, activation='linear', name='latent')(x)

        return Model(inputs=input_layer, outputs=latent, name='encoder')

    def create_decoder(self, latent_dim, output_shape):
        input_layer = tf.keras.Input(shape=(latent_dim,))
        
        # Calculate the shape after the encoder's final pooling
        # After 6 pooling operations (each divides by 2), and assuming input shape like (2, length, 1)
        # We need to calculate what the spatial dimensions would be
        height_after_pooling = output_shape[0] // (2**6)  # 6 pooling layers
        width_after_pooling = output_shape[1] // (2**6)   # 6 pooling layers
        
        # If dimensions become 0, set minimum of 1
        height_after_pooling = max(1, height_after_pooling)
        width_after_pooling = max(1, width_after_pooling)
        
        # Dense layer to expand latent vector
        x = Dense(height_after_pooling * width_after_pooling * 48, activation='relu')(input_layer)
        x = Reshape((height_after_pooling, width_after_pooling, 48))(x)
        
        # Decoder stacks - mirror the encoder but with upsampling
        x = self.decoder_stack(x, 16)
        x = self.decoder_stack2(x, 32) 
        x = self.decoder_stack2(x, 64)
        x = self.decoder_stack2(x, 128)
        x = self.decoder_stack2(x, 256)
        x = self.decoder_stack2(x, 512)
        
        # Final reconstruction layer
        output = Conv2D(output_shape[-1], 3, activation='linear', padding='same', name='reconstruction')(x)
        
        return Model(inputs=input_layer, outputs=output, name='decoder')

    def res_unit(self, x, dim, n):
        for _ in range(n):
            u = Conv2D(dim, 2, activation ='relu', padding='same')(x)
            u = Conv2D(dim, 2, activation='linear', padding='same')(u)  # Fixed: apply to u, not x
            # skip connection
            x = concatenate([u, x])
        return x
    def res_stack(self, x, dim):
        '''
        function that creates a residual stack for the model

        INPUT PARAMETERS
        x: layer to connect to
        dim: size of stack
        '''

        s = Conv2D(dim, 1, activation='linear', padding='same')(x)
        s = self.res_unit(s, dim, 2)
        s = MaxPooling2D(2, padding='same')(s)

        return s
    
    def decoder_stack(self, x, dim):
        '''
        function that creates a decoder stack for reconstruction

        INPUT PARAMETERS
        x: layer to connect to
        dim: size of stack (number of filters)
        '''
        # Upsample first
        x = Conv2DTranspose(dim, 2, strides=2, activation='linear', padding='same')(x)
        
        # Apply residual units
        s = Conv2D(dim, 1, activation='linear', padding='same')(x)
        s = self.decoder_res_unit(s, dim, 2)
        
        return s

    def decoder_stack2(self, x, dim):
        '''
        function that creates a decoder stack for reconstruction

        INPUT PARAMETERS
        x: layer to connect to
        dim: size of stack (number of filters)
        '''
        # Upsample first
        x = Conv2DTranspose(dim, 2, strides=(1,2), activation='linear', padding='same')(x)
        
        # Apply residual units
        s = Conv2D(dim, 1, activation='linear', padding='same')(x)
        s = self.decoder_res_unit(s, dim, 2)
        
        return s
    
    def decoder_res_unit(self, x, dim, n):
        '''
        Decoder residual unit - similar to res_unit but for decoder
        '''
        for _ in range(n):
            u = Conv2D(dim, 2, activation='relu', padding='same')(x)
            u = Conv2D(dim, 2, activation='linear', padding='same')(u)
            # skip connection
            x = concatenate([u, x])
        return x
        
    def call(self, inputs):
        '''
        Forward pass through the autoencoder
        '''
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded
    
    def encode(self, inputs):
        '''
        Get the latent representation
        '''
        return self.encoder(inputs)
    
    def decode(self, latent):
        '''
        Reconstruct from latent representation
        '''
        return self.decoder(latent)
        


In [16]:
model = Autoencoder(input_shape=(2, 1024, 1), output_shape=(2, 1024, 1))

In [17]:
# Instantiate and compile the autoencoder model
model = Autoencoder(input_shape=(2, 1024, 1), latent_dim=64)

# Compile the model
model.compile(
    optimizer='adam',
    loss='mse',  # Mean Squared Error for reconstruction
    metrics=['mae']  # Mean Absolute Error as additional metric
)

print("Model compiled successfully!")
print(f"Input shape: {model.encoder.input.shape}")
print(f"Latent dimension: {model.encoder.output.shape}")
print(f"Output shape: {model.decoder.output.shape}")

Model compiled successfully!
Input shape: (None, 2, 1024, 1)
Latent dimension: (None, 64)
Output shape: (None, 2, 1024, 1)


In [21]:
# Training configuration
EPOCHS = 20
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
LEARNING_RATE = 0.001

# Set up callbacks for better training
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

print("Training configuration set up!")
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Validation split: {VALIDATION_SPLIT}")
print(f"Learning rate: {LEARNING_RATE}")

Training configuration set up!
Epochs: 20
Batch size: 32
Validation split: 0.2
Learning rate: 0.001


In [19]:
# Data preprocessing and normalization
print("Original data shape:", X_train.shape)
print("Data type:", X_train.dtype)
print("Data range: [{:.6f}, {:.6f}]".format(X_train.min(), X_train.max()))

# Normalize the data to [-1, 1] range for better training
x_train_normalized = X_train.astype(np.float32)

# Optional: Apply normalization (uncomment if needed)
# x_train_normalized = (x_train_normalized - x_train_normalized.mean()) / x_train_normalized.std()

# For autoencoder training, input and target are the same (reconstruction task)
x_target = x_train_normalized.copy()

print("Preprocessed data shape:", x_train_normalized.shape)
print("Target data shape:", x_target.shape)
print("Normalized data range: [{:.6f}, {:.6f}]".format(x_train_normalized.min(), x_train_normalized.max()))

Original data shape: (2720, 2, 1024, 1)
Data type: float32
Data range: [-5.261979, 5.802701]
Preprocessed data shape: (2720, 2, 1024, 1)
Target data shape: (2720, 2, 1024, 1)
Normalized data range: [-5.261979, 5.802701]


In [22]:
# Train the autoencoder
print("Starting autoencoder training...")
print("="*50)

# Start training
history = model.fit(
    x_train_normalized,           # Input data
    x_target,                     # Target data (same as input for autoencoder)
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    callbacks=callbacks,
    verbose=1,
    shuffle=True
)

print("Training completed!")
print("="*50)

Starting autoencoder training...
Epoch 1/20
68/68 [==============================] - 68s 1s/step - loss: 0.5135 - mae: 0.6058 - val_loss: 0.5111 - val_mae: 0.6039 - lr: 0.0010
Epoch 2/20
68/68 [==============================] - 68s 1s/step - loss: 0.5135 - mae: 0.6058 - val_loss: 0.5111 - val_mae: 0.6039 - lr: 0.0010
Epoch 2/20
68/68 [==============================] - 67s 994ms/step - loss: 0.5135 - mae: 0.6058 - val_loss: 0.5110 - val_mae: 0.6039 - lr: 0.0010
Epoch 3/20
68/68 [==============================] - 67s 994ms/step - loss: 0.5135 - mae: 0.6058 - val_loss: 0.5110 - val_mae: 0.6039 - lr: 0.0010
Epoch 3/20
68/68 [==============================] - 67s 992ms/step - loss: 0.5135 - mae: 0.6058 - val_loss: 0.5110 - val_mae: 0.6039 - lr: 0.0010
Epoch 4/20
68/68 [==============================] - 67s 992ms/step - loss: 0.5135 - mae: 0.6058 - val_loss: 0.5110 - val_mae: 0.6039 - lr: 0.0010
Epoch 4/20
68/68 [==============================] - 67s 988ms/step - loss: 0.5134 - mae: 0.6058 -

In [ ]:
# Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 5))

# Plot training & validation loss
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# Plot training & validation MAE
plt.subplot(1, 3, 2)
plt.plot(history.history['mae'], label='Training MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Model MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.legend()
plt.grid(True)

# Plot learning rate (if ReduceLROnPlateau was triggered)
plt.subplot(1, 3, 3)
if 'lr' in history.history:
    plt.plot(history.history['lr'], label='Learning Rate')
    plt.title('Learning Rate')
    plt.xlabel('Epoch')
    plt.ylabel('LR')
    plt.legend()
    plt.grid(True)
else:
    plt.text(0.5, 0.5, 'No LR history available', 
             ha='center', va='center', transform=plt.gca().transAxes)
    plt.title('Learning Rate')

plt.tight_layout()
plt.show()

# Print final metrics
print(f"Final training loss: {history.history['loss'][-1]:.6f}")
print(f"Final validation loss: {history.history['val_loss'][-1]:.6f}")
print(f"Final training MAE: {history.history['mae'][-1]:.6f}")
print(f"Final validation MAE: {history.history['val_mae'][-1]:.6f}")

In [ ]:
# Evaluate the model and show reconstruction examples
print("Evaluating autoencoder performance...")

# Take a few samples for reconstruction testing
n_samples = 5
test_indices = np.random.choice(len(x_train_normalized), n_samples, replace=False)
test_samples = x_train_normalized[test_indices]

# Get reconstructions
reconstructions = model.predict(test_samples)
latent_representations = model.encode(test_samples)

print(f"Test samples shape: {test_samples.shape}")
print(f"Reconstructions shape: {reconstructions.shape}")
print(f"Latent representations shape: {latent_representations.shape}")

# Calculate reconstruction errors
mse_errors = np.mean((test_samples - reconstructions)**2, axis=(1,2,3))
mae_errors = np.mean(np.abs(test_samples - reconstructions), axis=(1,2,3))

print("\nReconstruction Errors:")
print("="*30)
for i in range(n_samples):
    print(f"Sample {i+1}:")
    print(f"  MSE: {mse_errors[i]:.6f}")
    print(f"  MAE: {mae_errors[i]:.6f}")

print(f"\nAverage MSE: {np.mean(mse_errors):.6f}")
print(f"Average MAE: {np.mean(mae_errors):.6f}")

In [ ]:
# Visualize original vs reconstructed IQ signals
plt.figure(figsize=(15, 10))

for i in range(min(3, n_samples)):  # Show first 3 samples
    # I component (real part)
    plt.subplot(3, 2, 2*i + 1)
    plt.plot(test_samples[i, 0, :, 0], label='Original I', alpha=0.7)
    plt.plot(reconstructions[i, 0, :, 0], label='Reconstructed I', alpha=0.7)
    plt.title(f'Sample {i+1} - I Component')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.grid(True)
    
    # Q component (imaginary part)
    plt.subplot(3, 2, 2*i + 2)
    plt.plot(test_samples[i, 1, :, 0], label='Original Q', alpha=0.7)
    plt.plot(reconstructions[i, 1, :, 0], label='Reconstructed Q', alpha=0.7)
    plt.title(f'Sample {i+1} - Q Component')
    plt.xlabel('Time')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()

# Show latent space statistics
print("\nLatent Space Analysis:")
print("="*30)
print(f"Latent dimension: {latent_representations.shape[1]}")
print(f"Latent mean: {np.mean(latent_representations):.6f}")
print(f"Latent std: {np.std(latent_representations):.6f}")
print(f"Latent min: {np.min(latent_representations):.6f}")
print(f"Latent max: {np.max(latent_representations):.6f}")

In [23]:
# Save the trained model
model_save_path = 'autoencoder_model'
encoder_save_path = 'encoder_model' 
decoder_save_path = 'decoder_model'

print("Saving trained models...")

# Save complete autoencoder
model.save(model_save_path)
print(f"Complete autoencoder saved to: {model_save_path}")

# Save encoder and decoder separately
model.encoder.save(encoder_save_path)
print(f"Encoder saved to: {encoder_save_path}")

model.decoder.save(decoder_save_path)
print(f"Decoder saved to: {decoder_save_path}")

# Save training history
import pickle
with open('training_history.pkl', 'wb') as f:
    pickle.dump(history.history, f)
print("Training history saved to: training_history.pkl")

print("\nAll models and history saved successfully!")
print("="*50)

# Print model summaries
print("\nFinal Model Architecture:")
print("="*30)
print("\nEncoder Summary:")
model.encoder.summary()
print("\nDecoder Summary:")
model.decoder.summary()

Saving trained models...


INFO:tensorflow:Assets written to: autoencoder_model\assets


INFO:tensorflow:Assets written to: autoencoder_model\assets


Complete autoencoder saved to: autoencoder_model


INFO:tensorflow:Assets written to: encoder_model\assets


INFO:tensorflow:Assets written to: encoder_model\assets


Encoder saved to: encoder_model


INFO:tensorflow:Assets written to: decoder_model\assets


INFO:tensorflow:Assets written to: decoder_model\assets


Decoder saved to: decoder_model
Training history saved to: training_history.pkl

All models and history saved successfully!

Final Model Architecture:

Encoder Summary:
Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 2, 1024, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d_120 (Conv2D)            (None, 2, 1024, 512  1024        ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_121 (Conv2D)  